In [12]:
import json
from collections import OrderedDict
from typing import Callable
from pprint import pprint

In [13]:
class DefaultOrderedDict(OrderedDict):
    # Source: http://stackoverflow.com/a/6190500/562769
    def __init__(self, default_factory=None, *a, **kw):
        if (default_factory is not None and
           not isinstance(default_factory, Callable)):
            raise TypeError('first argument must be callable')
        OrderedDict.__init__(self, *a, **kw)
        self.default_factory = default_factory

    def __getitem__(self, key):
        try:
            return OrderedDict.__getitem__(self, key)
        except KeyError:
            return self.__missing__(key)

    def __missing__(self, key):
        if self.default_factory is None:
            raise KeyError(key)
        self[key] = value = self.default_factory()
        return value

    def __reduce__(self):
        if self.default_factory is None:
            args = tuple()
        else:
            args = self.default_factory,
        return type(self), args, None, None, self.items()

    def copy(self):
        return self.__copy__()

    def __copy__(self):
        return type(self)(self.default_factory, self)

    def __deepcopy__(self, memo):
        import copy
        return type(self)(self.default_factory,
                          copy.deepcopy(self.items()))

    def __repr__(self):
        return 'OrderedDefaultDict(%s, %s)' % (self.default_factory,
                                               OrderedDict.__repr__(self))

In [14]:
with open('./test.json', 'r') as f:
    data = json.load(f)

In [15]:
len(data['tests'])

20

In [16]:
pprint(list(data['tests'][0].keys()))

['endPos',
 'log',
 'timeStart',
 'biome',
 'traceback',
 'status',
 'timeTaken',
 'error',
 'startPos',
 'world',
 'timeEnd']


In [17]:
total = len(data['tests'])
sucesses = 0

errors = DefaultOrderedDict(int)
erro_biome = DefaultOrderedDict(int)
times = []
for i in data['tests']:
    if i['status'] == True:
        sucesses += 1
        times.append(i['timeTaken'])
    else:
        errors[i['error']] += 1
        erro_biome[i['biome']] += 1

print(f'Sucesses: {sucesses}/{total} ({((sucesses/total)*100):.2f}%)')
pprint(dict(errors))
pprint(dict(erro_biome))

Sucesses: 5/20 (25.00%)
{'No crafting table found': 3,
 'Timeout': 1,
 'Walk: Cannot find a valid path to the objective': 1,
 'Walk: Cannot follow the path of the pathfinder': 10}
{'Beach': 1,
 'Deep Ocean': 3,
 'Extreme Hills': 1,
 'Ocean': 1,
 'Plains': 4,
 'River': 2,
 'Savanna': 1,
 'Swampland': 2}


In [21]:
print(min(times))
print(sum(times)/len(times))
print(max(times))

103.093
129.0586
154.535


In [19]:
e = [d for d in data['tests'] if d['status'] == False][0]

In [20]:
print(e['error'])
print(e['traceback'])

Walk: Cannot follow the path of the pathfinder
@D:\GitHub\autocraft\macros\libs\walk.lua:515 Walk: Cannot follow the path of the pathfinder
stack traceback:
	D:\GitHub\autocraft\macros\libs\walk.lua:515: in function 'walkTo'
	D:\GitHub\autocraft\macros\libs\gathering/miner.lua:61: in function 'goToMinePlace'
	D:\GitHub\autocraft\macros\libs\gathering/miner.lua:263: in function 'mine'
	D:\GitHub\autocraft\macros\libs\farm.lua:6: in function '?'
	D:\GitHub\autocraft\macros\libs\crafting.lua:210: in function 'craft'
	D:\GitHub\autocraft\macros\libs\crafting.lua:225: in function 'craft'
	D:\GitHub\autocraft\macros\libs\command.lua:155: in function <D:\GitHub\autocraft\macros\libs\command.lua:144>
	[Java]: in function 'pcall'
	D:\GitHub\autocraft\macros\libs\command.lua:50: in function <D:\GitHub\autocraft\macros\libs\command.lua:49>
	[Java]: in ?
